In [1]:
!wget https://raw.githubusercontent.com/dhhse/fanfics4hack/main/data/FanFics_Metadata_full.json

--2021-03-09 14:06:17--  https://raw.githubusercontent.com/dhhse/fanfics4hack/main/data/FanFics_Metadata_full.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40558938 (39M) [text/plain]
Saving to: ‘FanFics_Metadata_full.json.1’

FanFics_Metadata_fu 100%[===================>]  38.68M  53.0MB/s    in 0.7s    

2021-03-09 14:06:18 (53.0 MB/s) - ‘FanFics_Metadata_full.json.1’ saved [40558938/40558938]



In [15]:
!wget http://rusvectores.org/static/models/rusvectores4/fasttext/araneum_none_fasttextcbow_300_5_2018.tgz

--2021-03-09 19:34:19--  http://rusvectores.org/static/models/rusvectores4/fasttext/araneum_none_fasttextcbow_300_5_2018.tgz
Resolving rusvectores.org (rusvectores.org)... 116.203.104.23
Connecting to rusvectores.org (rusvectores.org)|116.203.104.23|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2691248108 (2.5G) [application/x-gzip]
Saving to: ‘araneum_none_fasttextcbow_300_5_2018.tgz’

araneum_none_fastte 100%[===================>]   2.51G  10.7MB/s    in 4m 19s  

2021-03-09 19:38:39 (9.91 MB/s) - ‘araneum_none_fasttextcbow_300_5_2018.tgz’ saved [2691248108/2691248108]



In [18]:
!tar xzf araneum_none_fasttextcbow_300_5_2018.tgz

python3


In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder

from copy import deepcopy 
from typing import List, Tuple

In [2]:
df = pd.read_json("FanFics_Metadata_full.json")

df.shape

(19258, 25)

In [3]:
# если пропущенных значений много -- дропаем колонку, иначе строку
count_columns_null = df.isnull().sum(axis=0).to_dict()

for column in count_columns_null:
    if count_columns_null[column] > 1000:
        df.drop(column, axis=1, inplace=True)

df.dropna(inplace=True)
df.shape

(18804, 20)

In [4]:
# оставляем колонки со строками и числами
filtered_columns = []
for column in df.columns:
    if type(df[column][0]) not in [list, dict]:
        filtered_columns.append(column)

df = df[filtered_columns]

df.shape

(18804, 11)

In [5]:
df.head()

,ID,Title,Views,Readers,Likes,Collections,Comments,Genre,Rating,Status,Description
0,139006,"The Secret Diary of Fleur Delacour, aged 16 34",8110.0,307.0,0.0,18.0,12.0,гет,PG-13,В процессе,Это своего рода эксперимент. Большинство людей...
1,137435,"Рабочие будни Уолдена МакНейра, ликвидатора оп...",4331.0,509.0,2.0,38.0,157.0,джен,General,Закончен,В жизни ликвидатора опасных существ проблемы с...
2,2306,Яблоко Гесперид,77454.0,418.0,5.0,32.0,66.0,гет,R,Закончен,Министерство издает Закон о чистоте крови. Дра...
3,103183,Однажды в другой вселенной,7595.0,54.0,0.0,16.0,15.0,гет,R,Закончен,Сборник совершенно не связанных между собой за...
4,98951,Мыс альбатросов,53937.0,695.0,5.0,41.0,867.0,гет,R,В процессе,Иногда затишье перед бурей страшнее разыгравше...


#### Черновики

In [6]:
def select_columns(df) -> Tuple[List[str], List[str]]:
    """
    По количеству уникальных значений в колонке
    функция отбирает какие колонки закодировать one-hot,
    а какие нужно векторизовать.
    Возвращает 2 списка названий колонок: для one-hot и для векторизации
    """
    one_hot_columns = []
    vect_columns = []
    unique_value_threshold = df.shape[0] * 0.01

    for column in df.columns:
        if type(df[column][0]) is str:
            if df[column].unique().shape[0] < unique_value_threshold:
                one_hot_columns.append(column)
            else:
                vect_columns.append(column)

    return one_hot_columns, vect_columns



one_hot_columns, vect_columns = select_columns(df)

In [ ]:
def apply_vectorizer() -> pd.DataFrame:
    """
    применяет к отфильтрованным колонкам векторизацию
    и возвращает датафрейм
    """
    concatenated_columns = df[vect_columns[0]]
    for column in vect_columns[1:]:
        concatenated_columns = concatenated_columns + " " + df[column]

    vect = CountVectorizer()
    values = vect.fit_transform(concatenated_columns).toarray()
    return pd.DataFrame(values, columns=vect.get_feature_names())

In [8]:
def apply_one_hot() -> pd.DataFrame:
    """
    применяет к отфильтрованным колонкам one-hot кодирование
    и возвращает датафрейм
    """
    encoder = OneHotEncoder(sparse=False)
    values = encoder.fit_transform(df[one_hot_columns])
    return pd.DataFrame(values, columns=encoder.get_feature_names())

#### Класс

In [77]:
import gensim
fast_text_model = gensim.models.KeyedVectors.load('araneum_none_fasttextcbow_300_5_2018.model')

True

In [87]:
class DFWrapper:
    def __init__(self, df: pd.DataFrame, target: str, tfidf=False, fasttext=False) -> None:
        """
        Принимает датафрейм, где все колонки содержат или числа, или строки, 
        таргет, который будет использован для классификации
        """
        self.df = df.drop(target, axis=1)
        self.target = df[target]
        if not tfidf and not fasttext:
            self.vectorizer = CountVectorizer
        elif tfidf:
            self.vectorizer = TfidfVectorizer
        elif fasttext:
            self.vectorizer = "fasttext"
        self.__modify_df()

    @property
    def features(self) -> pd.DataFrame:
        """
        Возвращает подготовленный к классификации датафрейм,
        где строковые колонки или закодированы one-hot, или векторизованы
        """
        return self.df

    def fit(self, model):
        """
        Обучение модели
        """
        m = model()
        m.fit(self.df, self.target)
        return m

    def __select_columns(self) -> Tuple[List[str], List[str]]:
        """
        По количеству уникальных значений в колонке
        функция отбирает какие колонки закодировать one-hot,
        а какие нужно векторизовать.
        Возвращает 2 списка названий колонок: для one-hot и для векторизации
        """
        one_hot_columns = []
        vect_columns = []
        unique_value_threshold = self.df.shape[0] * 0.01

        for column in self.df.columns:
            if type(self.df[column][0]) is str:
                if self.df[column].unique().shape[0] < unique_value_threshold:
                    one_hot_columns.append(column)
                else:
                    vect_columns.append(column)
        return one_hot_columns, vect_columns

    def __apply_one_hot(self, one_hot_columns: List[str]) -> pd.DataFrame:
        """
        Принимает список названий колонок и применяет к ним one-hot кодирование.
        Возвращает датафрейм
        """
        encoder = OneHotEncoder(sparse=False)
        values = encoder.fit_transform(self.df[one_hot_columns])
        return pd.DataFrame(values, columns=encoder.get_feature_names())

    def __apply_vectorizer(self, vect_columns: List[str]) -> pd.DataFrame:
        """
        Принимает список названий колонок и применяет к ним векторизацию.
        Возвращает датафрейм
        """
        concatenated_columns = self.df[vect_columns[0]]
        for column in vect_columns[1:]:
            concatenated_columns = concatenated_columns + " " + self.df[column]

        vect = self.vectorizer()
        values = vect.fit_transform(concatenated_columns).toarray()
        return pd.DataFrame(values, columns=vect.get_feature_names())

    def __make_embeddings(self, string: str) -> np.array:
        """
        Принимает строку и возвращает усредненный вектор эмбеддинга фасттекста
        """
        res = []
        # конечно нужно бы токенизировать по-хорошему
        for word in string.split():
            try:
                res.append(fast_text_model.word_vec(word))
            except AttributeError:
                continue
        return np.array(res).mean(axis=0)

    def __apply_fasttext(self, vect_columns: List[str]) -> None:
        """
        Принимает список названий колонок и переводит их в эмбеддинги фасттекста.
        """
        for column in vect_columns:
            self.df[column] = self.df[column].apply(self.__make_embeddings)

    def __modify_df(self) -> None:
      """
      Применяет к исходному датафрейму one-hot кодирование и векторизацию.
      """
      one_hot_columns, vect_columns = self.__select_columns()
      one_hot_df = self.__apply_one_hot(one_hot_columns)
      if self.vectorizer != "fasttext":
          vectorized_df = self.__apply_vectorizer(vect_columns)
          self.df = pd.concat((self.df, one_hot_df, vectorized_df), axis=1)
          self.df.drop(one_hot_columns + vect_columns, axis=1, inplace=True)
      else:
          self.__apply_fasttext(vect_columns)
          self.df = pd.concat((self.df, one_hot_df), axis=1)
          self.df.drop(one_hot_columns, axis=1, inplace=True)

In [88]:
new_df = DFWrapper(df.head(1000), "Genre", tfidf=False, fasttext=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:83: RuntimeWarning: Mean of empty slice.
